# Imports & File Load

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42
# import requests
# import json
import re
# import os
# import time
# from bs4 import BeautifulSoup
from collections import Counter
%matplotlib inline

In [2]:
df = pd.read_csv("sources/judges_careersCurrentFirst.csv")
df.head(3)

,nid,jid,Last Name,First Name,Middle Name,Suffix,Birth Month,Birth Day,Birth Year,Birth City,...,Degree (3),Degree Year (3),School (4),Degree (4),Degree Year (4),School (5),Degree (5),Degree Year (5),Professional Career,Other Nominations/Recess Appointments
0,1393211,3275,Peelle,Stanton,Judkins,,2.0,11.0,1843,Richmond,...,NaN,1866,NaN,NaN,NaN,NaN,NaN,NaN,"U.S. Army corporal and second lieutenant, 1861...",NaN
1,1393011,3235,Barney,Samuel,Stebbins,,1.0,31.0,1846,Hartford,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"High school teacher, Hartford, Wisconsin, 1869...",NaN
2,1393056,3244,Atkinson,George,Wesley,,6.0,29.0,1845,Charleston,...,LL.B.,1874,NaN,NaN,NaN,NaN,NaN,NaN,"Assistant superintendent of public schools, Ka...",NaN


# File Investigation

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3694 entries, 0 to 3693
Columns: 201 entries, nid to Other Nominations/Recess Appointments
dtypes: float64(56), int64(2), object(143)
memory usage: 3.6+ MB


In [4]:
original_size = df.memory_usage().sum()
original_size

3827024

In [5]:
# df.dtypes
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3694 entries, 0 to 3693
Data columns (total 201 columns):
nid                                      int64
jid                                      int64
Last Name                                object
First Name                               object
Middle Name                              object
Suffix                                   object
Birth Month                              float64
Birth Day                                float64
Birth Year                               object
Birth City                               object
Birth State                              object
Death Month                              float64
Death Day                                float64
Death Year                               float64
Death City                               object
Death State                              object
Gender                                   object
Race or Ethnicity                        object
Court Type (1)           

In [6]:
# df.profile_report()

# Basic Clean
- make easy column headers
- remove the circa abbreviation everywhere 
- convert separated birth/death info to united field
- convert all string dates into datetime
- convert year-only dates into datetime
- convert various strings to categories
- make a seat id that's not tied to individuals

In [7]:
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('/', '-')
df.columns = df.columns.str.replace('\(|\)|,', '')
df.replace('ca. ','',regex=True,inplace=True)

In [8]:
df['Birth_Month'] = pd.to_numeric(df['Birth_Month'], errors='coerce').fillna(value=9).astype(int)
df['Birth_Day'] = pd.to_numeric(df['Birth_Day'], errors='coerce').fillna(value=30).astype(int)
df['Birth_Year'] = pd.to_numeric(df['Birth_Year'], errors='coerce').fillna(value=1677).astype(int)
df['Birth_Date'] = df.Birth_Year.map(str).astype(str).str.zfill(4) + '-' + df.Birth_Month.map(str).astype(str).str.zfill(2) + '-' + df.Birth_Day.map(str).astype(str).str.zfill(2)
df['Death_Month'] = pd.to_numeric(df['Death_Month'], errors='coerce').fillna(value=9).astype(int)
df['Death_Day'] = pd.to_numeric(df['Death_Day'], errors='coerce').fillna(value=30).astype(int)
df['Death_Year'] = pd.to_numeric(df['Death_Year'], errors='coerce').fillna(value=1677).astype(int)
df['Death_Date'] = df.Death_Year.map(str).astype(str).str.zfill(4) + '-' + df.Death_Month.map(str).astype(str).str.zfill(2) + '-' + df.Death_Day.map(str).astype(str).str.zfill(2)

In [9]:
df.Death_Date.head()

0    1928-09-04
1    1919-12-31
2    1925-04-04
3    1930-03-28
4    1920-06-27
Name: Death_Date, dtype: object

In [10]:
forceDT = [
    'Birth_Date','Death_Date',
    'Recess_Appointment_Date_1','Nomination_Date_1','Committee_Referral_Date_1','Hearing_Date_1','Committee_Action_Date_1','Confirmation_Date_1','Commission_Date_1','Service_as_Chief_Judge_Begin_1','Service_as_Chief_Judge_End_1','2nd_Service_as_Chief_Judge_Begin_1','2nd_Service_as_Chief_Judge_End_1','Senior_Status_Date_1','Termination_Date_1',
    'Recess_Appointment_Date_2','Nomination_Date_2','Committee_Referral_Date_2','Hearing_Date_2','Committee_Action_Date_2','Confirmation_Date_2','Commission_Date_2','Service_as_Chief_Judge_Begin_2','Service_as_Chief_Judge_End_2','2nd_Service_as_Chief_Judge_Begin_2','2nd_Service_as_Chief_Judge_End_2','Senior_Status_Date_2','Termination_Date_2',
    'Recess_Appointment_Date_3','Nomination_Date_3','Committee_Referral_Date_3','Hearing_Date_3','Committee_Action_Date_3','Confirmation_Date_3','Commission_Date_3','Service_as_Chief_Judge_Begin_3','Service_as_Chief_Judge_End_3','2nd_Service_as_Chief_Judge_Begin_3','2nd_Service_as_Chief_Judge_End_3','Senior_Status_Date_3','Termination_Date_3',
    'Recess_Appointment_Date_4','Nomination_Date_4','Committee_Referral_Date_4','Hearing_Date_4','Committee_Action_Date_4','Confirmation_Date_4','Commission_Date_4','Service_as_Chief_Judge_Begin_4','Service_as_Chief_Judge_End_4','2nd_Service_as_Chief_Judge_Begin_4','2nd_Service_as_Chief_Judge_End_4','Senior_Status_Date_4','Termination_Date_4',
    'Recess_Appointment_Date_5','Nomination_Date_5','Committee_Referral_Date_5','Hearing_Date_5','Committee_Action_Date_5','Confirmation_Date_5','Commission_Date_5','Service_as_Chief_Judge_Begin_5','Service_as_Chief_Judge_End_5','2nd_Service_as_Chief_Judge_Begin_5','2nd_Service_as_Chief_Judge_End_5','Senior_Status_Date_5','Termination_Date_5',
    'Recess_Appointment_Date_6','Nomination_Date_6','Committee_Referral_Date_6','Hearing_Date_6','Committee_Action_Date_6','Confirmation_Date_6','Commission_Date_6','Service_as_Chief_Judge_Begin_6','Service_as_Chief_Judge_End_6','2nd_Service_as_Chief_Judge_Begin_6','2nd_Service_as_Chief_Judge_End_6','Senior_Status_Date_6','Termination_Date_6'
]
df[forceDT] = df[forceDT].apply(pd.to_datetime)

In [11]:
df['Degree_Year_1'] = pd.to_datetime(df['Degree_Year_1'],format='%Y', errors='coerce')
df['Degree_Year_2'] = pd.to_datetime(df['Degree_Year_2'],format='%Y', errors='coerce')
df['Degree_Year_3'] = pd.to_datetime(df['Degree_Year_3'],format='%Y', errors='coerce')
df['Degree_Year_4'] = pd.to_datetime(df['Degree_Year_4'],format='%Y', errors='coerce')
df['Degree_Year_5'] = pd.to_datetime(df['Degree_Year_5'],format='%Y', errors='coerce')

In [24]:
df['Full_Name'] = df.First_Name + ' ' + df.Middle_Name + ' ' + df.Last_Name
df.Full_Name.replace(r'\s{2,4}',' ',regex=True,inplace=True)
df.Full_Name.head(3)

0    Oliver Wendell Holmes
1        William Rufus Day
2      William Henry Moody
Name: Full_Name, dtype: object

In [68]:
#MAKE USEFUL SEAT INFO
# df['Seat_Code_1'] = df.Seat_ID_1.str.slice(0,-2)
# df.Seat_Code_1.replace(r'(.*)D(\d\d)',r'\1-\2',regex=True,inplace=True)
# df.Seat_Code_1.replace(r'(CA)(\d{1,2}|DC)(\d\d)',r'\1\2-\3',regex=True,inplace=True)
# df.Seat_Code_1.replace(r'(SC)T(\d\d)',r'\1-\2',regex=True,inplace=True)
# df.Seat_Code_1.value_counts()

# df.Seat_ID_1.str.extract(r'(.*)(\d\d)(\d\d)')
# df['Seat_Code_1'] = df.Court_Abbrev_Website_1.str.replace(r' ',r'') + '-' + df.Seat_ID_1.str.extract(r'.*(\d\d)\d\d').dropna()

df['Seat_ID_SeatNum_1'] = df.Seat_ID_1.str.extract(r'.*(\d\d)\d\d').dropna()
df['Seat_ID_Code_1'] = df.Court_Abbrev_Website_1.str.replace(r' ',r'') + '-'
df['Seat_Code_1'] = df.Seat_ID_Code_1 + df.Seat_ID_SeatNum_1
df.Seat_Code_1.value_counts()









# SeatNumbers = df.Seat_ID_1.str.extract(r'.*(\d\d)\d\d').dropna()
# SeatCodes = df.Court_Abbrev_Website_1.str.replace(r' ',r'') + '-'
# df['Seat_Code_1'] = df.SeatCodes + SeatNumbers
# df.Seat_Code_1.value_counts()

SC-04         19
SC-01         18
03-NJ-02      17
03-PA-W-01    16
01-RI-01      14
SC-03         13
SC-05         13
02-VT-01      13
01-ME-01      12
09-CCA-03     12
SC-02         12
08-NE-01      12
03-DE-01      12
SC-07         12
01-CCA-03     11
SC-09         11
05-TX-E-01    11
03-CCA-02     11
08-AR-W-02    11
05-TX-W-01    11
06-CCA-01     10
01-NH-01      10
03-PA-E-01    10
02-CCA-03     10
02-CT-01      10
07-CCA-03     10
09-MT-01      10
11-AL-S-02     9
04-MD-01       9
07-IN-W-01     9
              ..
05-TX-W-11     1
09-CA-S-26     1
08-CCA-02      1
07-IN-N-05     1
09-CA-S-24     1
04-VA-E-12     1
01-MA-12       1
03-PA-E-02     1
11-AL-N-11     1
09-CA-S-01     1
10-FL-S-22     1
04-SC-11       1
04-SC-03       1
10-FL-M-12     1
03-PA-M-02     1
01-MA-13       1
09-AZ-06       1
07-IN-E-05     1
11-CCA-05      1
09-CA-N-05     1
10-FL-S-23     1
11-CCA-08      1
09-AZ-14       1
05-TX-S-15     1
01-PR-06       1
05-CCA-09      1
06-KY-E-06     1
04-CCA-15     

In [13]:
df.memory_usage().sum() / original_size * 100

118.14652847748015

In [14]:
forceCat = [
    'Gender','Race_or_Ethnicity','Birth_City','Birth_State','Death_City','Death_State',
    'Court_Type_1','Court_Name_1','Appointment_Title_1','Appointing_President_1','Party_of_Appointing_President_1','Reappointing_President_1','Party_of_Reappointing_President_1','ABA_Rating_1','Judiciary_Committee_Action_1','Senate_Vote_Type_1','Termination_1',
    'Court_Type_2','Court_Name_2','Appointment_Title_2','Appointing_President_2','Party_of_Appointing_President_2','Reappointing_President_2','Party_of_Reappointing_President_2','ABA_Rating_2','Judiciary_Committee_Action_2','Senate_Vote_Type_2','Termination_2',
    'Court_Type_3','Court_Name_3','Appointment_Title_3','Appointing_President_3','Party_of_Appointing_President_3','Reappointing_President_3','Party_of_Reappointing_President_3','ABA_Rating_3','Judiciary_Committee_Action_3','Senate_Vote_Type_3','Termination_3',
    'Court_Type_4','Court_Name_4','Appointment_Title_4','Appointing_President_4','Party_of_Appointing_President_4','Reappointing_President_4','Party_of_Reappointing_President_4','ABA_Rating_4','Judiciary_Committee_Action_4','Senate_Vote_Type_4','Termination_4',
    'Court_Type_5','Court_Name_5','Appointment_Title_5','Appointing_President_5','Party_of_Appointing_President_5','Reappointing_President_5','Party_of_Reappointing_President_5','ABA_Rating_5','Judiciary_Committee_Action_5','Senate_Vote_Type_5','Termination_5',
    'Court_Type_6','Court_Name_6','Appointment_Title_6','Appointing_President_6','Party_of_Appointing_President_6','Reappointing_President_6','Party_of_Reappointing_President_6','ABA_Rating_6','Judiciary_Committee_Action_6','Senate_Vote_Type_6','Termination_6',
    'School_1','Degree_1','School_2','Degree_2','School_3','Degree_3','School_4','Degree_4','School_5','Degree_5'
]
df[forceCat] = df[forceCat].astype('category')
df.memory_usage().sum() / original_size * 100

93.02408346537675

In [ ]:
# df = df.drop(columns='url')

In [20]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3401 entries, 0 to 3400
Data columns (total 210 columns):
nid                                      int64
jid                                      int64
Last_Name                                object
First_Name                               object
Middle_Name                              object
Suffix                                   object
Birth_Month                              int32
Birth_Day                                int32
Birth_Year                               int32
Birth_City                               category
Birth_State                              category
Death_Month                              int32
Death_Day                                int32
Death_Year                               int32
Death_City                               category
Death_State                              category
Gender                                   category
Race_or_Ethnicity                        category
Court_Type_1            

# Join Key Tables

In [15]:
columns = ['Pres_Num','Pres_Name','Pres_State_Abbrev']
pres_df = pd.read_csv("sources/key_presidents.csv", usecols=columns)
df = df.merge(pres_df, left_on='Appointing_President_1', right_on='Pres_Name')
df = df.rename({'Pres_Num':'Appointing_President_Num_1', 'Pres_State_Abbrev':'Appointing_President_State_1'}, axis=1)
del df['Pres_Name']
df.head()

,nid,jid,Last_Name,First_Name,Middle_Name,Suffix,Birth_Month,Birth_Day,Birth_Year,Birth_City,...,School_5,Degree_5,Degree_Year_5,Professional_Career,Other_Nominations-Recess_Appointments,Birth_Date,Death_Date,Full_Name,Appointing_President_Num_1,Appointing_President_State_1
0,1393211,3275,Peelle,Stanton,Judkins,,2,11,1843,Richmond,...,NaN,NaN,NaT,"U.S. Army corporal and second lieutenant, 1861...",NaN,1843-02-11,1928-09-04,Stanton Judkins Peelle,26,NY
1,1393011,3235,Barney,Samuel,Stebbins,,1,31,1846,Hartford,...,NaN,NaN,NaT,"High school teacher, Hartford, Wisconsin, 1869...",NaN,1846-01-31,1919-12-31,Samuel Stebbins Barney,26,NY
2,1393056,3244,Atkinson,George,Wesley,,6,29,1845,Charleston,...,NaN,NaN,NaT,"Assistant superintendent of public schools, Ka...",NaN,1845-06-29,1925-04-04,George Wesley Atkinson,26,NY
3,1382341,1082,Holmes,Oliver,Wendell,Jr.,3,8,1841,Boston,...,NaN,NaN,NaT,"U.S. Army lieutenant, 20th Massachusetts Volun...",President issued recess appointment to Supreme...,1841-03-08,1935-03-06,Oliver Wendell Holmes,26,NY
4,1379896,588,Day,William,Rufus,,4,17,1849,Ravenna,...,NaN,NaN,NaT,"Private practice, Canton, Ohio, 1872-1897, 189...",Received recess appointment to U.S. District C...,1849-04-17,1923-07-09,William Rufus Day,26,NY


In [16]:
columns = ['Court_Circuit','Court_Name_Long','Court_Abbrev_Bluebook','Court_Abbrev_Website','State_Abbrev']
pres_df = pd.read_csv("sources/key_courts.csv", usecols=columns)
df = df.merge(pres_df, left_on='Court_Name_1', right_on='Court_Name_Long')
df = df.rename({'Court_Circuit':'Court_Circuit_1','Court_Abbrev_Bluebook':'Court_Abbrev_Bluebook_1','Court_Abbrev_Website':'Court_Abbrev_Website_1','State_Abbrev':'Court_State_1'}, axis=1)
del df['Court_Name_Long']
df.head()

,nid,jid,Last_Name,First_Name,Middle_Name,Suffix,Birth_Month,Birth_Day,Birth_Year,Birth_City,...,Other_Nominations-Recess_Appointments,Birth_Date,Death_Date,Full_Name,Appointing_President_Num_1,Appointing_President_State_1,Court_Circuit_1,Court_Abbrev_Bluebook_1,Court_Abbrev_Website_1,Court_State_1
0,1382341,1082,Holmes,Oliver,Wendell,Jr.,3,8,1841,Boston,...,President issued recess appointment to Supreme...,1841-03-08,1935-03-06,Oliver Wendell Holmes,26,NY,Fed,U.S.,SC,NaN
1,1379896,588,Day,William,Rufus,,4,17,1849,Ravenna,...,Received recess appointment to U.S. District C...,1849-04-17,1923-07-09,William Rufus Day,26,NY,Fed,U.S.,SC,NaN
2,1385276,1672,Moody,William,Henry,,12,23,1853,Newbury,...,NaN,1853-12-23,1917-07-02,William Henry Moody,26,NY,Fed,U.S.,SC,NaN
3,1384126,1441,Lurton,Horace,Harmon,,2,26,1844,Newport,...,NaN,1844-02-26,1914-07-12,Horace Harmon Lurton,27,OH,Fed,U.S.,SC,NaN
4,1389696,2567,White,Edward,Douglass,,11,3,1845,Lafourche Parish,...,NaN,1845-11-03,1921-05-19,Edward Douglass White,27,OH,Fed,U.S.,SC,NaN


In [17]:
df.memory_usage().sum() / original_size * 100

89.46476426591524

In [18]:
df[['Appointing_President_State_1','Court_Circuit_1','Court_Abbrev_Bluebook_1','Court_Abbrev_Website_1','Court_State_1']] = df[['Appointing_President_State_1','Court_Circuit_1','Court_Abbrev_Bluebook_1','Court_Abbrev_Website_1','Court_State_1']].astype('category')
df.memory_usage().sum() / original_size * 100

88.3833234387869

# Basic Research

### Most Appointments

In [ ]:
df.Appointing_President_1.value_counts()

### Qualifications
- ABA Rating
- Committee Recommendations
- Termination Types

In [ ]:
df.ABA_Rating_1.value_counts()

In [ ]:
df[df.ABA_Rating_1.str.contains("Not", na=False)]['Appointing_President_1'].value_counts()

In [ ]:
df[df.ABA_Rating_1.str.contains("Not", na=False)][['Full_Name','ABA_Rating_1','Appointing_President_1','Confirmation_Date_1']].sort_values(by='Confirmation_Date_1', ascending=False)

In [ ]:
df[df.ABA_Rating_2.str.contains("Not", na=False)][['Full_Name','ABA_Rating_2','Appointing_President_2','Confirmation_Date_2']].sort_values(by='Confirmation_Date_2', ascending=False)

In [ ]:
df.Judiciary_Committee_Action_1.value_counts()

In [ ]:
df[~df.Judiciary_Committee_Action_1.str.contains("\(favorably", na=False)][['Full_Name','Judiciary_Committee_Action_1','Appointing_President_1','Confirmation_Date_1']].sort_values(by='Confirmation_Date_1', ascending=False).dropna().head(60)

In [21]:
df.Termination_1.value_counts()

Death                               1540
Retirement                           283
Resignation                          213
Recess Appointment-Not Confirmed      16
Impeachment & Conviction               8
Abolition of Court                     0
Name: Termination_1, dtype: int64

In [ ]:
df[df.Termination_1.str.contains("Impeachment", na=False)][['Full_Name','Termination_1','Appointing_President_1','Confirmation_Date_1']].sort_values(by='Confirmation_Date_1', ascending=False)

In [ ]:
df[df.Termination_1.str.contains("Recess", na=False)][['Full_Name','Termination_1','Appointing_President_1','Recess_Appointment_Date_1']].sort_values(by='Recess_Appointment_Date_1', ascending=False)

### Declined & Rejected Appointments

In [ ]:
df[df['Other_Nominations-Recess_Appointments'].str.contains("(?i)Declined", na=False)][['Full_Name','Other_Nominations-Recess_Appointments','Appointing_President_1','Confirmation_Date_1']].sort_values(by='Confirmation_Date_1', ascending=False)

In [ ]:
df[df['Other_Nominations-Recess_Appointments'].str.contains("(?i)Rejected", na=False)][['Full_Name','Other_Nominations-Recess_Appointments','Appointing_President_1','Confirmation_Date_1']].sort_values(by='Confirmation_Date_1', ascending=False)

In [ ]:
df['Other_Nomination'] = df["Other_Nominations-Recess_Appointments"].str.findall(r"\b(nominated.*\d\d\d\d); .*", flags = re.I).dropna(0)
df['Other_Rejection'] = df["Other_Nominations-Recess_Appointments"].str.findall(r"\b(rejected.*\d\d\d\d).*", flags = re.I).dropna(0)
df[df['Other_Nominations-Recess_Appointments'].str.contains(r"rejected", na=False, flags = re.I)][['Full_Name','Other_Nomination','Other_Rejection','Appointing_President_1','Confirmation_Date_1']].sort_values(by='Confirmation_Date_1', ascending=False)

In [ ]:
df.loc[df.Party_of_Reappointing_President_1 != df.Party_of_Appointing_President_1][['Full_Name','Appointing_President_1','Reappointing_President_1','Recess_Appointment_Date_1','Nomination_Date_1','Confirmation_Date_1']].dropna().sort_values(by='Confirmation_Date_1', ascending=False)

### Demographics Breakdowns

In [ ]:
df.groupby(by='Party_of_Appointing_President_1').Gender.value_counts(1)

In [ ]:
df.groupby(by='Party_of_Appointing_President_1').Race_or_Ethnicity.value_counts(1)

In [ ]:
# Rcount = df.groupby(by='Party_of_Appointing_President_1').Race_or_Ethnicity.value_counts()
Rcount = df.Race_or_Ethnicity.value_counts()
# sns.barplot(data=df, x='Rcount', y='Rcount', col='Party_of_Appointing_President_1', height=3)
# sns.barplot(y=Rcount.index, x=Rcount.values, col='Party_of_Appointing_President_1', height=3)
sns.barplot(ydata=df, y='Race_or_Ethnicity'.index, x='Race_or_Ethnicity'.values, col='Party_of_Appointing_President_1', height=3)
# sns.barplot(data=df, x='Rcount', y='Rcount', col='Party_of_Appointing_President_1', height=3)

In [ ]:
df.groupby(by='Party_of_Appointing_President_1').Race_or_Ethnicity.value_counts().plot(kind='bar',figsize=(18, 5))

### Professional Background

In [ ]:
AVERAGR AGE AT TIME OF APPOINTMENT
LENGTH OF SERVICE
LENGTH OF PRIOR SERVICE
FROM PRESIDENTS STATE



In [ ]:
df.Professional_Career.head()

In [ ]:
df.Professional_Career = df.Professional_Career.astype(str)
# Counter(" ".join(df.Professional_Career).lower().split()).most_common(60)
pd.Series(' '.join(df.Professional_Career).lower().split()).value_counts().head(60)

In [ ]:
df[df.Professional_Career.str.contains('army|navy|marine|force', na=False, flags = re.I)].groupby(by='Appointing_President_1').size().sort_values(ascending=False).head(15)

In [ ]:
search_terms = ['professor','lecturer','faculty','dean']
df[df.Professional_Career.str.contains('|'.join(search_terms),na=False,case=False)].groupby(by='Appointing_President_1').size().sort_values(ascending=False).head(15)

In [ ]:
###   BIGRAMS / NGRAMS  ???

# df['Prof_Repeats'] = df.Professional_Career.str.findall(r"(?=(\b\w+\W+\w+\W+\w+))",flags=re.I).dropna(0)
# df.Prof_Repeats.head()
# # pd.Series(' '.join(df.Prof_Repeats).lower().split()).value_counts().head(60)
# # Counter(" ".join(df.Prof_Repeats).lower().split()).most_common(60)

# bigrams = []
# for idx, row in df.iterrows():
#     lst = row.Professional_Career.split(' ')
#     bigrams.append([(lst[x].strip(), lst[x+1].strip()) for x in range(len(lst)-1)])
# print(bigrams)


# def bigram(row):
#     lst = row['Professional_Career'].split(' ')
#     return [(lst[x].strip(), lst[x+1].strip()) for x in range(len(lst)-1)]
# bigrams = df.apply(lambda row: bigram(row), axis=1)
# print(bigrams.tolist())

In [ ]:
### Gave up, found a ngrams library

from sklearn.feature_extraction.text import CountVectorizer
word_vectorizer = CountVectorizer(ngram_range=(2,2), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(df.Professional_Career)
frequencies = sum(sparse_matrix).toarray()[0]
pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency']).sort_values(by='frequency', ascending=False).head(20)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
word_vectorizer = CountVectorizer(ngram_range=(3,4), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(df.Professional_Career)
frequencies = sum(sparse_matrix).toarray()[0]
pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency']).sort_values(by='frequency', ascending=False).head(20)

### Education

In [ ]:
lastreadlaw = [df[df['School_1'] == 'Read law'].Degree_Year_1.max(),
               df[df['School_2'] == 'Read law'].Degree_Year_3.max(), 
               df[df['School_3'] == 'Read law'].Degree_Year_3.max(),
               df[df['School_4'] == 'Read law'].Degree_Year_4.max(), 
               df[df['School_5'] == 'Read law'].Degree_Year_5.max()
              ]
lastreadlaw

In [ ]:
allschool_df = df[['Full_Name','Appointing_President_1','Party_of_Appointing_President_1']]
allschool_df['Schl1'] = df.School_1 + '-777777777-' + df.Degree_1.fillna('nun') + '-999999999-' + df.Degree_Year_1.astype(str).fillna('nun')
allschool_df['Schl2'] = df.School_2 + '-777777777-' + df.Degree_2.fillna('nun') + '-999999999-' + df.Degree_Year_2.astype(str).fillna('nun')
allschool_df['Schl3'] = df.School_3 + '-777777777-' + df.Degree_3.fillna('nun') + '-999999999-' + df.Degree_Year_3.astype(str).fillna('nun')
allschool_df['Schl4'] = df.School_4 + '-777777777-' + df.Degree_4.fillna('nun') + '-999999999-' + df.Degree_Year_4.astype(str).fillna('nun')
allschool_df['Schl5'] = df.School_5 + '-777777777-' + df.Degree_5.fillna('nun') + '-999999999-' + df.Degree_Year_5.astype(str).fillna('nun')
allschool_df

In [ ]:
allschool_df = allschool_df.melt(id_vars =['Full_Name','Appointing_President_1','Party_of_Appointing_President_1'], value_vars =['Schl1','Schl2','Schl3','Schl4','Schl5'], var_name='Degree_Number', value_name='School')

In [ ]:
allschool_df.Degree_Number.replace(r'Schl',r'',regex=True,inplace=True)
allschool_df['Degree'] = allschool_df.School.str.extract('-777777777-(.*)-999999999-', expand=False)
allschool_df['Degree_Year'] = allschool_df.School.str.extract('.*-999999999-(.*)$', expand=False)
allschool_df['School'] = allschool_df.School.str.extract(r'(.*)-777777777-.*', expand=False)
allschool_df['DYear'] = allschool_df.Degree_Year.str.extract('^(\d\d\d\d).*', expand=False)
allschool_df.replace('nun',np.nan,inplace=True)
allschool_df.Degree_Year = allschool_df.Degree_Year.astype('datetime64[ns]') 

In [ ]:
allschool_df.head(15)

In [ ]:
allschool_df['School_Neat'] = allschool_df['School']
allschool_df.School_Neat.replace(r'.*\(now (.*)\)',r'\1',regex=True,inplace=True)
allschool_df.replace({'School_Neat':{
    'College of':'Cof',
    ' College':'',
    'University of':'Uof',
    ' University':'',
    ' Department':'',
    ' Faculty':'',
    ' School':'',
    ' Center':'',
    ' of Law':'',
    ' Law':'',
    ' of Jurisprudence':'',
    ' of Business':'',
    ' Business':'',
    ' Administration':'',
    ' Wharton':'',
    ' Gould':'',
    ', Boalt Hall':'',
    ', Paul M. Hebert':'',
    '-Kansas City':'',
    ' -- Newark':'',
    ' -- Camden':'',
    '  ':' '
}},regex=True,inplace=True)
allschool_df.School_Neat.replace(r'\s?Cof','',regex=True,inplace=True)
allschool_df.School_Neat.replace(r'\s?$','',regex=True,inplace=True)
allschool_df.School_Neat.replace(r',$','',regex=True,inplace=True)
allschool_df.School_Neat.value_counts().head(60)

In [ ]:
allschool_df.School_Neat.value_counts().head(30).plot(kind='barh', figsize=(18, 8), y='School_Neat').invert_yaxis()

In [ ]:
allschool_df.groupby(['DYear']).School_Neat.count().plot(figsize=(18, 8))


# allschool_df.School_Neat.value_counts().head(30).plot(kind='barh', figsize=(18, 8), y='School_Neat').invert_yaxis()
# allschool_df.School_Neat.value_counts().head(30).groupby(by='DYear').plot(kind='barh', figsize=(18, 8)).invert_yaxis()

# allschool_df.plot(x='DYear', y='School_Neat', figsize=(18, 5), lw=0.5)
# allschool_df.groupby(by='DYear').School_Neat.value_counts(1)

In [ ]:
allschool_df.groupby(by='Party_of_Appointing_President_1').School_Neat.value_counts(1)

In [ ]:
allschool_df.Degree.value_counts().head(20)

In [ ]:
allschool_df['Degree_Neat'] = allschool_df['Degree']
# allschool_df.School_Neat.replace(r'.*\(now (.*)\)',r'\1',regex=True,inplace=True)
# allschool_df.replace({'School_Neat':{
#     'College of':'Cof',
#     ' College':'',
#     'University of':'Uof',
#     ' University':'',
#     ' Department':'',
#     ' Faculty':'',
#     ' School':'',
#     ' Center':'',
#     ' of Law':'',
#     ' Law':'',
#     ' of Jurisprudence':'',
#     ' of Business':'',
#     ' Business':'',
#     ' Administration':'',
#     ' Wharton':'',
#     ' Gould':'',
#     ', Boalt Hall':'',
#     ', Paul M. Hebert':'',
#     '-Kansas City':'',
#     ' -- Newark':'',
#     ' -- Camden':'',
#     '  ':' '
# }},regex=True,inplace=True)
# allschool_df.School_Neat.replace(r'\s?Cof','',regex=True,inplace=True)
# allschool_df.School_Neat.replace(r'\s?$','',regex=True,inplace=True)
# allschool_df.School_Neat.replace(r',$','',regex=True,inplace=True)
# allschool_df.School_Neat.value_counts().head(60)

In [ ]:
allschool_df.groupby(by='DYear').Degree.value_counts().unstack().plot(figsize=(18, 5), kind='line')

In [ ]:
allschool_df.Degree.value_counts().plot(figsize=(18, 5), kind='line', x='DYear', y='Degree')


In [ ]:
allschool_df[['Degree', 'DYear']]
# value_counts().plot(figsize=(18, 5), kind='line', x='DYear', y='Degree')

In [ ]:
allschool_df.groupby(by='DYear').Degree.value_counts().plot(figsize=(18, 5))
# allschool_df.groupby(by='DYear').Degree.count().plot(figsize=(18, 5))
# allschool_df.groupby(['DYear','Degree']).count().unstack().plot.line(x='DYear', y='Degree', figsize=(18, 5), lw=0.5)

# allschool_df.Degree.value_counts().plot.line(x='Degree', y='DYear', figsize=(18, 5), lw=0.5)
# allschool_df.groupby(by='DYear').School_Neat.value_counts(1)

In [ ]:
allschool_df.to_csv('allschool.csv', index=False)

In [ ]:
df[df['School_1'].isin(['Harvard University', 'Yale University', 'Brown University', 'Columbia University', 'Princeton University'])]

### Confirmation Dates

In [ ]:
df['Confirmation_Duration_Total'] = df.Confirmation_Date_1 - df.Nomination_Date_1
df.Confirmation_Duration_Total.sort_values(ascending=False).describe()

In [ ]:
df.groupby(by='Appointing_President_1').Confirmation_Duration_Total.max().sort_values(ascending=False)

In [ ]:
df[(df.Appointing_President_1.str.contains('Barack')) & (df.Termination_Date_1 < '20160115')][['Full_Name','Court_Type_1','Confirmation_Date_1','Termination_1','Termination_Date_1','Court_Type_2']].sort_values(by=['Confirmation_Date_1','Termination_Date_1'],ascending=[0,1])

In [ ]:
strg = dt.strftime('%Y-%m-%d %H-%M-%S')

In [69]:
df.to_csv('judges_pandaed.csv', index=False)